In [1]:
%%writefile PLW.py
import numpy as np
from scipy.optimize import minimize


def myPLWLS(y,x,attr1=x,attr_intercept=0,mse=1,costfn=1):
    """run the PLWLS optimization function, projecting x onto y with correspondence distances attr1
    return scipy minimize object"""
    print('mse=',mse,'costfn=',costfn)
    if np.allclose(np.ones([attr1.shape[0],]),attr1[:,0]):#fix problems with differencing column of 1's
        attr=attr1[:,1:]
    else: attr=attr1
    n,k=np.shape(x)
    diffstack=Attr_To_Diffstck(attr,attr_intercept)
    j=diffstack.shape[2]
    wt_try0=np.zeros([j,])
    return minimize(PLWLStryMSE,wt_try0,args=(y,x,diffstack,mse,costfn),method='Nelder-Mead')

def Attr_To_Diffstck(attr,attr_intercept):
    """takes an n*k matrix of correspondence attributes and returns an n*n*k array of differences"""
    n,j=np.shape(attr); j+=1 #plus 1 to go with the column of 1's added after differencing
    attr_tile=np.tile(attr,(n,1,1))# stack the attributes matrix (nxk array) n times
    diffstack=np.transpose(attr_tile,(1,0,2))-attr_tile#transpose the z and x dimensions (listed as z,x,y)
    if attr_intercept==0
        return diffstack
    if attr_intercept==1:
        return np.concatenate((np.ones([n,n,1]),diffstack), axis=2)


def Reg_wts_stack(corrparams,diffstack,norm_wt_stack=0):
    #return np.sum(((2**corrparams*diffstack)**2),axis=2)**-0.5)
    #weighted, squared, summed across wt parameter, j, rooted and flipped
    regwt_stack=np.sum(np.exp((corrparams*diffstack)**2),axis=2)**(-0.5)
    if norm_wt_stack==1:
        return regwt_stack/(np.sum(regwt_stack**2)**.5/regwt_stack.shape[0])#normalize the weights seems to help convergence
    if norm_wt_stack==0:
        return regwt_stack
    
def PLWLScastMSE(corrparams,attr1,y,x,norm_wt_stack=0,attr_intercept=0):
    #use corrparams, correspondence attributes, known y and x to calculate mse for the estimator/corrparams
    if np.allclose(np.ones([attr1.shape[0],]),attr1[:,0]):#fix problems with differencing column of 1's
        attr=attr1[:,1:]
    else: attr=attr1
    n,k=np.shape(x)
    regwt_stack=Reg_wts_stack(corrparams,Attr_To_Diffstck(attr,attr_intercept),norm_wt_stack)
    
    x_wt_stck=np.tile(regwt_stack.reshape(n,n,1),(1,1,k))*np.tile(x,(n,1,1))
    y_wt_stck=regwt_stack.reshape(n,n,1)*np.tile(y.reshape(n,1),(n,1,1))
    x_wt_stckT=np.transpose(x_wt_stck,(0,2,1))#transpose X aross the stack for x'x
    Qi=np.linalg.inv(x_wt_stckT@x_wt_stck)
    Bstack=Qi@x_wt_stckT@y_wt_stck
    #print('first 5 params=',Bstack[0:5,:,:])
    
                             
    ystack=np.tile(y.reshape(n,1),(n,1,1))
    xstack=np.tile(x,(n,1,1))
    err=(ystack-xstack@Bstack)#errors are not calculated using weighted y and x
    
    yhat_stack=xstack@Bstack                         
    yhats=yhat_stack.diagonal().diagonal()
    err_diag=err.diagonal().diagonal()
    mse=np.sum(err_diag**2)/err_diag.size
    '''p=figure()
    p.circle(x[:,1],y)
    p.square(x[:,1],yhats)
    p.show()'''
    #print('mse_cast=',mse)
    return mse
    

    
def PLWLStryMSE(corrparams,y,x,diffstack,MSE=1,costfn=1,norm_wt_stack=0):
    #print(x.shape)
    print('corrparams=',corrparams)
    n,k=np.shape(x)
    j=np.shape(corrparams)[0]
    #print(k,j)
    regwt_stack=Reg_wts_stack(corrparams,diffstack,norm_wt_stack)
    x_wt_stck=np.tile(regwt_stack.reshape(n,n,1),(1,1,k))*np.tile(x,(n,1,1))#the stack of weights is broadcast and multiplied by stack of x.(z,x,y) indexing
    y_wt_stck=regwt_stack.reshape(n,n,1)*np.tile(y.reshape(n,1),(n,1,1))#the stack of weights is broadcast and multiplied by stack of y.
    x_wt_stckT=np.transpose(x_wt_stck,(0,2,1))#transpose X aross the stack
    Qi=np.linalg.inv(x_wt_stckT@x_wt_stck)
    #print(y_wt_stck.shape)
    Bstack=Qi@x_wt_stckT@y_wt_stck #use weighted data to calculate gls parameters
    #print('first params=',Bstack[0,:,:])
    ystack=np.tile(y.reshape(n,1),(n,1,1))
    xstack=np.tile(x,(n,1,1))
    err=(ystack-xstack@Bstack)#errors are not calculated using weighted y and x
    if costfn==1: cost_err=err**2-regwt_stack**(-2) #squared error prediction
    if costfn==2: cost_err=err
    corr_err=err**2-regwt_stack**(-2)
    if MSE==1:
        out=np.sum(cost_err**2)/cost_err.size
        print('{}MSE all='.format(costfn),out)
        return out
    if MSE==2:
        wt_cost_err=(cost_err)*regwt_stack
        out=np.sum(wt_cost_err**2)/wt_cost_err.size
        print('{}MSE wt_all='.format(costfn),out)
        return out
    if MSE==3:
        diag_err=cost_err.diagonal().diagonal()
        print('diag',diag_err.shape)
        out=np.sum(diag_err**2)/diag_err.size
        print('{}MSE n_only='.format(costfn),out)
        return out
    
def PLWLSpredict(corrparams,attr1,y,x,attr1_val,y_val,x_val,norm_wt_stack=0):
    #use corrparams, correspondence attributes, known y and x to calculate mse for the estimator/corrparams
    if np.allclose(np.ones([attr1.shape[0],]),attr1[:,0]):#fix problems with differencing column of 1's
        attr=attr1[:,1:]
    else: attr=attr1
    if np.allclose(np.ones([attr1_val.shape[0],]),attr1_val[:,0]):#fix problems with differencing column of 1's
        attr_val=attr1_val[:,1:]    
    else: attr_val=attr1_val
    n,k=np.shape(x)
    nv,k1=np.shape(x_val)
    j=np.shape(attr)[1]+1 #plus 1 to go with the column of 1's added after differencing
    ctr_attr_stck=np.tile(attr_val,(n,1,1))# stack the centered attributes matrix (nxk array) n times
    diffstack0=np.transpose(ctr_attr_stck,(1,0,2))-np.tile(attr,(nv,1,1))#transpose the z and x dimensions (listed as z,x,y)
    diffstack=np.concatenate((np.ones([nv,n,1]),diffstack0), axis=2) ##so diffstack is shape nv,n,j
    
    regwt_stack=Reg_wts_stack(corrparams,diffstack,norm_wt_stack)
    x_wt_stck=np.tile(regwt_stack.reshape(nv,n,1),(1,1,k))*np.tile(x,(nv,1,1))
    y_wt_stck=regwt_stack.reshape(nv,n,1)*np.tile(y.reshape(n,1),(nv,1,1))
    x_wt_stckT=np.transpose(x_wt_stck,(0,2,1))#transpose X aross the stack for x'x
    Qi=np.linalg.inv(x_wt_stckT@x_wt_stck)
    Bstack=Qi@x_wt_stckT@y_wt_stck
    #print('first 5 params=',Bstack[0:5,:,:])
                                 
    ystack=np.tile(y_val.reshape(nv,1),(nv,1,1))
    xstack=np.tile(x_val,(nv,1,1))
    err=(ystack-xstack@Bstack)#errors are not calculated using weighted y and x
    
    yhat_stack=xstack@Bstack                         
    yhats=yhat_stack.diagonal().diagonal()
    err_diag=err.diagonal().diagonal()
    mse=np.sum(err_diag**2)/err_diag.size
    '''p=figure()
    p.circle(x[:,1],y)
    p.square(x[:,1],yhats)
    p.show()'''
    #print('mse_cast=',mse)
    return mse    


Writing PLW.py


In [2]:
%%writefile reg.py
import numpy as np

class reg():
    

        def myreg(self,y,x):
            # a simple regression that prints its own mse and returns estimated parameters
            q=np.linalg.inv(np.matmul(x.T,x))
            self.b=np.matmul(np.matmul(q,x.T),y)
            self.mse=np.sum((y-x@self.b)**2)/y.shape[0]

        def myregstdz(self,y,x): 
            [n,k]=np.shape(x)
            onecol=0
            xfull=x
            if np.allclose(np.ones([n,1]),x[:,0]):#fix problems with standardizing column of 1's
                x=x[:,1:]
                onecol=1;
            xmean=np.mean(x,axis=0)
            xvar=np.var(x,axis=0,ddof=1)#subtracting xmean isn't necessary, as it happens anyways
            onevec=np.ones([n,1])
            yvar=np.var(y, ddof=1)
            sy=np.power(yvar,.5)
            sx=np.power(xvar,.5)
            ymean=np.mean(y)        
            xstdz=(1/sx)*(x-xmean)
            ystdz=(1/sy)*(y-ymean)
            Q=np.linalg.inv(np.matmul(xstdz.T,xstdz))
            bhat1=np.matmul(np.matmul(Q,xstdz.T),ystdz)
            bhat=sy*bhat1*(1/sx)#undo beta coef. by multiplying each by Sy/Sx
            if onecol==1:
                bhat_int=ymean-np.sum(sy*xmean*bhat1*(1/sx))
                bhat=np.concatenate([[bhat_int],bhat])
            self.mse=np.sum((y-xfull@bhat)**2)/y.shape[0]
            self.b=bhat

            def myregpredict(y,x,yval,xval):
    # a simple regression that prints its own mse and returns estimated parameters
    q=np.linalg.inv(np.matmul(x.T,x))
    bhat=np.matmul(np.matmul(q,x.T),y)
    mse=np.sum((yval-xval@bhat)**2)/yval.shape[0]
    #print('mse=',mse)
    return mse

Writing reg.py


In [3]:
%%writefile data_gen.py
import random
import numpy as np

class data_gen():
    '''generates numpy arrays of random training or validation for model: y=xb+e or variants
    '''
    def __init__(self, xtrain_size=(200,5), ftype='linear', xval_size='same', sparsity=0, xvar=1, xmean=0, evar=1, betamax=10):
        k_1=200
        n=5
        self.xtrain=
        
    
        xtall=xmean
        xwide=xvar
        spreadx=np.random.randint(xwide, size=(1,k_1))+1#random row vector to multiply by each random column of x to allow s.d. upto 5
        shiftx=np.random.randint(0,xtall, size=(1,k_1))-xtall/2#random row vector to add to each column of x
        randx=np.random.randn(n,k_1)
        self.x = np.concatenate((np.ones((n,1)),shiftx+spreadx*randx),axis=1)
        #generate error~N(0,1)
        self.e=np.random.randn(n)*evar**.5
        

        #make beta integer, non-zero
        self.b=(np.random.randint(betamax, size=(k_1+1,))+1)*(2*np.random.randint(2, size=(k_1+1,))-np.ones(k_1+1,)) #if beta is a random integer, it could be 0
        #make a simple y for testing
        self.y=np.matmul(self.x, self.b)+self.e

Writing data_gen.py
